In [ ]:
from pynta.postprocessing import *
from ase.visualize import view
from ase.io import read
from ase.io.trajectory import Trajectory
import json

# Surface Information and Reference Energies

In [ ]:
path = "../pynta/testing_data/Ru0001_fischer_tropsch_3_17_25"

#Surface information
metal = "Ru" #specify the metal
facet = "hcp0001" #specify the facet
slab = read(os.path.join(path,"slab.xyz"))
nslab = len(slab)
site_density = get_site_density(slab,metal,facet)

#Site Information
cas = SlabAdsorptionSites(slab, facet,allow_6fold=False,composition_effect=False,
                            label_sites=True,
                            surrogate_metal=metal)
sites = cas.get_sites()
site_adjacency = cas.get_neighbor_site_list()

#Reference Energies
c_ref,o_ref,h_ref,n_ref,finished_atoms = get_reference_energies(os.path.join(path,"Adsorbates"),nslab)

# Analyze Species

In [ ]:
ad_path = os.path.join(path,"Adsorbates","O[Pt]") #specify the path to the Pynta species directory

In [ ]:
spcs = get_species(ad_path,os.path.split(ad_path)[0],metal,facet,slab,sites,site_adjacency,nslab,c_ref=c_ref,o_ref=o_ref,h_ref=h_ref,
               n_ref=n_ref) #all configurations
minspc = spcs[min({k:v for k,v in spcs.items() if v.valid},key=lambda x: spcs[x].energy)] #lowest energy valid configuration

In [ ]:
print("Relative Energies of Configurations: {} [eV]".format({k:v.energy - minspc.energy for k,v in spcs.items()}))
print("Validity of Configurations: {}".format({k:v.valid for k,v in spcs.items()}))

In [ ]:
#view(s.atoms for s in spcs.values()) #see all configurations

In [ ]:
#Individual species
spc = minspc #or spc = spcs['0'] etc.

In [ ]:
print("Frequencies: {} cm^-1".format(spc.vibdata.get_frequencies()))

In [ ]:
references_available = True #check if thermodynamic references were available and thus the thermochemistry is properly referenced
for elm in spc.mol.get_element_count().keys():
    if elm not in finished_atoms and elm != 'X': #we don't have appropriate references for this species
        references_available = False

In [ ]:
if references_available:
    print("Hf298: {} kcal/mol".format(spc.heat_of_formation_298K/4184.0))
    print("Sf298: {} cal/(mol K)".format(spc.entropy_of_formation_298K/4.184))

In [ ]:
#view(spc.atoms) #see individual configuration

In [ ]:
#Specific Thermodynamic Properties J-mol-K
T = 1000.0
if references_available:
    print("Gf(T={0}): {1} kcal/mol".format(T,spc.nasa.get_free_energy(T)/4184.0))
    print("Hf(T={0}): {1} kcal/mol".format(T,spc.nasa.get_enthalpy(T)/4184.0))
    print("Sf(T={0}): {1} cal/(mol K)".format(T,spc.nasa.get_entropy(T)/4.184))
    print("Cp(T={0}): {1} cal/(mol K)".format(T,spc.nasa.get_heat_capacity(T)/4.184))

# Analyze Transition State


In [ ]:
ts_path = os.path.join(path,"TS3") #specify the TS directory to analyze

In [ ]:
plot_eharm(ts_path,Eharmtol=3.0,Eharmfiltertol=30.0) #plot the harmonic energies of the transition state guesses and cutoffs 

In [ ]:
kinetics = get_kinetics(ts_path,os.path.join(os.path.split(ts_path)[0],"Adsorbates"),metal,facet,slab,sites,site_adjacency,nslab,site_density,
                c_ref=c_ref,o_ref=o_ref,h_ref=0.0,n_ref=0.0)
minkinind = min({k:v for k,v in kinetics.items() if v.valid},key=lambda x: kinetics[x].barrier_f) #index of lowest energy valid configuration

In [ ]:
print(kinetics[minkinind].reaction_str) #Reaction String

In [ ]:
print("Forward Barriers: {} [eV]".format({k:v.barrier_f for k,v in kinetics.items()}))
print("Reverse Barriers: {} [eV]".format({k:v.barrier_r for k,v in kinetics.items()}))
print("Automated Validity Determination: {}".format({k:v.valid for k,v in kinetics.items()}))

In [ ]:
#Individual Transition States/Kinetics
ind = minkinind #'7' etc.
kin = kinetics[ind]

In [ ]:
print("Frequencies: {} cm^-1".format(kin.transition_state.vibdata.get_frequencies()))

In [ ]:
kin.arr_f #Forward rate coefficient

In [ ]:
kin.arr_r #Reverse rate coefficient

In [ ]:
#Specific Thermodynamic Properties J-mol-K
T = 1000.0
print("dGrxn(T={0}): {1} kcal/mol".format(T,kin.get_gibbs_energy_reaction(T)/4184.0))
print("dHrxn(T={0}): {1} kcal/mol".format(T,kin.get_enthalpy_reaction(T)/4184.0))
print("dSrxn(T={0}): {1} cal/(mol K)".format(T,kin.get_entropy_reaction(T)/4.184))

In [ ]:
#Check TS Validity Information
kin.valid_info

In [ ]:
#Visualize TS
view(read(os.path.join(ts_path,ind,"opt.xyz")))

In [ ]:
#Visualize IRC1 
view(Trajectory(os.path.join(ts_path,ind,"irc_forward.traj")))

In [ ]:
#Visualize IRC2
view(Trajectory(os.path.join(ts_path,ind,"irc_reverse.traj")))

In [ ]:
#Visualize Imaginary Frequency
view(Trajectory(os.path.join(ts_path,ind,"vib.0.traj")))